In [3]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.optimize
from tqdm import tqdm

from visualisation.utils import *
from visualisation.sobol import *
from visualisation.emissionpathways import *
from carbontaxdamages.economics import *

## Calibration and model parameters

In [4]:
colors_SSPs = {
    'SSP1': colors_PBL[3],
    'SSP2': colors_PBL[1],
    'SSP3': colors_PBL[4],
    'SSP4': colors_PBL[2],
    'SSP5': colors_PBL[5]
}

### Calibration of mitigation costs

In [3]:
def baseline_consumption(SSP_lst, t_values_years_lst):
    # Baseline consumption is (100-21)% of baseline GDP
    return [(1-0.21) * GDP(year, SSP) for SSP, year in zip(SSP_lst, t_values_years_lst)]

def baseline_emissions_lst(SSP_lst, t_values_years_lst):
    return [baseline_emissions(year, SSP) for SSP, year in zip(SSP_lst, t_values_years_lst)]

def calc_consumption_loss(df):
    df = df.copy()
    df.loc[:,'baseline_consumption'] = baseline_consumption(df['SSP'], df['year'])
    df.loc[:,'baseline_emissions'] = baseline_emissions_lst(df['SSP'], df['year'])
    
    year0 = df['year'].min()
    
    df.loc[:,'NPV_consumption_diff'] = np.exp(-0.05 * (df['year'] - year0)) * (df['baseline_consumption'] - df['consumption'])
    df.loc[:,'NPV_baseline_consumption'] = np.exp(-0.05 * (df['year'] - year0)) * df['baseline_consumption']
    
    df = df.groupby('name').agg({
        'baseline_emissions': 'sum',
        'NPV_consumption_diff': 'sum',
        'NPV_baseline_consumption': 'sum',
        'CE': 'last',
        'SSP': 'first', 'damage': 'first', 'TCRE': 'first', 'cost_level': 'first', 'r': 'first', 'gamma': 'first'
    })
    
    df['baseline_emissions'] *= 5
    
    df['rel_CB'] = df['CE'] / df['baseline_emissions']
    
    df['consumption_loss'] = df['NPV_consumption_diff'] / df['NPV_baseline_consumption']
    
    return df

In [5]:
costs_ar5 = pd.read_csv("../AR5fig6.23.csv").rename(columns={'X-Value': 'x', 'Y-Value': 'y'})
costs_ar5['y'] /= 100 # Costs are in percentage points
costs_ar5_GE = costs_ar5[costs_ar5['ModelType'] == 'GE']

from_calibration_CL = pd.read_csv('calibration_consumption_losses.csv')
n = 10
from_calibration_CL.loc[:,'SSP'] = (['SSP1']*n + ['SSP2']*n + ['SSP3']*n + ['SSP4']*n + ['SSP5']*n) * int(len(from_calibration_CL)/(5*n))
from_calibration_CL['scenario'] = ['{}_{}_{}'.format(*a) for a in zip(from_calibration_CL['rho'], from_calibration_CL['beta'], from_calibration_CL['cost_level'])]

In [6]:
## Then, perform constrained linear regression such that every line intersects
## the y=0 line at the same point

f1 = lambda x: (x-1)**3

def f(a):
    return lambda x: a * f1(x)

formula = 'y ~ f1(x) - 1'

model = smf.quantreg(formula, costs_ar5_GE)
functions = {'p{:02.0f}'.format(q*100): f(*model.fit(q=q).params.values) for q in [0.05, 0.5, 0.95]}

In [11]:
fig = go.Figure()

# Plot AR5 cost points
fig.add_trace(go.Scatter(
    x=costs_ar5_GE['x'], y=costs_ar5_GE['y'], mode='markers',
    name='AR5 consumption<br>losses', marker={'color': 'rgba(0.5, 0.5, 0.5, 0.9)'},
    legendgroup='AR5'
))

percentile_colors = dict(zip(['p05', 'p50', 'p95'], px.colors.qualitative.Plotly))
symbols = dict(zip(['SSP1','SSP2', 'SSP3', 'SSP4', 'SSP5'], ['pentagon', 'square', 'diamond', 'triangle-up', 'star']))

showpoints = True

# Plot quantile regression results
for percentile in [95, 50, 5]:
    name = 'p{:02d}'.format(percentile)
    xvalues = np.linspace(0.1, 0.62, 100)
    fct = f(*model.fit(q=percentile/100).params.values)
    fig.add_trace(go.Scatter(
        x=xvalues, y=fct(xvalues), name='{}th percentile'.format(percentile),
        legendgroup='percentiles', line_color = percentile_colors[name], showlegend=(not showpoints)
    ))
    if showpoints:
        fig.add_trace(go.Scatter(
            x=[None], y=[None], name='{}th percentile'.format(percentile),
            legendgroup='percentiles', line_color = percentile_colors[name], mode='markers+lines'
        ))

        
if showpoints:
    
    for name, selection in from_calibration_CL[
        (from_calibration_CL['rho'] == 0.82) & (from_calibration_CL['beta'] == 2.0)
    ].groupby('cost_level'):
        
        symbols_SSP = [symbols[SSP] for SSP in selection['SSP']]
        
        fig.add_trace(go.Scatter(
            x=selection['cb'], y=selection['NPV'], legendgroup='percentiles', showlegend=False,
            mode='markers', marker_color=percentile_colors[name],
            marker_symbol = symbols_SSP
        ))
    
    for SSP, symbol in symbols.items():
        fig.add_trace(go.Scatter(
            x=[None], y=[None], legendgroup='symbols', name=SSP, mode='markers',
            marker={'symbol': symbol, 'color': '#000'}
        ))

fig.update_layout(
    xaxis_title='Cumulative CO<sub>2</sub> emissions (fraction of baseline)',
    yaxis={'title': 'Consumption loss (% baseline consumption)', 'tickformat': ',.0%'},
    width=950, height=450,
    margin={'t': 20, 'b': 20}, font_size=13,
    legend_y=0.5
    #legend_title='Quantile regression:'
)
fig.show()

In [12]:
fig.write_image('../Paper/img/calibration_AR5_costs' + ('_withpoints' if showpoints else '') + '.png', scale=2)

### SSPs

In [14]:
extended = False
years = np.linspace(2020, 2200 if extended else 2100, 100)
SSPs = ['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5']

fig = make_subplots(1,3, subplot_titles=('Emissions', 'GDP per capita', 'Population'), horizontal_spacing=0.08)
for SSP in SSPs:
    color = colors_SSPs[SSP]
    fig.append_trace(go.Scatter(x=years, y=baseline_emissions(years, SSP), line_color=color, name=SSP, legendgroup=SSP), 1, 1)
    fig.append_trace(go.Scatter(x=years, y=GDP(years, SSP) * 1e12 / population(years, SSP), line_color=color, showlegend=False, legendgroup=SSP), 1, 2)
    fig.append_trace(go.Scatter(x=years, y=population(years, SSP), line_color=color, showlegend=False, legendgroup=SSP), 1, 3)
fig.update_layout(
    legend_y=0.5, width=950, height=350, margin={'l': 20, 'r': 30, 'b': 30, 't': 30},
    yaxis1={'title': 'GtCO<sub>2</sub>/yr', 'range': [0, 117.29], 'title_standoff': 0},
    yaxis2={'title': 'US$/yr', 'title_standoff': 0, 'rangemode': 'tozero'},
    yaxis3={'range': [0, 1.496e+10 if extended else 13e9]}
)
fig.show()

In [15]:
fig.write_image('../Paper/img/SSP_baselines.png', scale=2)

### Damage functions

In [54]:
temperatures = np.linspace(0.5, 5.5, 100)
SSPs = ['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5']

fig = make_subplots(1,2, subplot_titles=('Long Run (with lag)', 'Short Run (no lag)'), horizontal_spacing=0.08)
for SSP in SSPs:
    color = colors_SSPs[SSP]
    fig.add_scatter(x=Burke_damage_data.loc[(1, SSP), 'temperature'].values, y=Burke_damage_data.loc[(1, SSP), 'damage'].values, line_color=color, name=f'BHM ({SSP})', legendgroup='BHM', row=1, col=1)
    fig.add_scatter(x=Burke_damage_data.loc[(0, SSP), 'temperature'].values, y=Burke_damage_data.loc[(0, SSP), 'damage'].values, line_color=color, name=f'BHM ({SSP})', legendgroup='BHM', showlegend=False, row=1, col=2)

for col in [1,2]:
    fig.add_scatter(x=temperatures, y=damages['damageHowardTotal'](temperatures), name='Howard Total', line={'color': '#999', 'dash': 'dot'}, showlegend=col==1, row=1, col=col)
    fig.add_scatter(x=temperatures, y=damages['damageDICE'](temperatures), name='DICE', line={'color': '#999', 'dash': 'dash'}, showlegend=col==1, row=1, col=col)
    fig.add_scatter(x=temperatures, y=damages['damageDICE'](temperatures), name='DICE', line={'color': '#999', 'dash': 'dash'}, showlegend=col==1, row=1, col=col)
    
    howard_p05, howard_p95 = 0.195, 1.293
    tippingpoints_factor = 1.25
    dmg_howard_extra = lambda a, temperatures: tippingpoints_factor * (a/100) * temperatures**2
    fig.add_scatter(
        x=np.concatenate([temperatures, temperatures[::-1]]), 
        y=np.concatenate([dmg_howard_extra(howard_p05, temperatures), dmg_howard_extra(howard_p95, temperatures[::-1])]),
        fill='toself', line={'color': 'rgba(0,0,0,0)', 'width': 0}, fillcolor='rgba(0.75,0.75,0.75,0.3)',
        name='Howard (p05-p95)', showlegend=col==1, row=1, col=col
    )
    
fig.update_yaxes(range=[-0.05, 0.8], title='GDP loss', tickformat='%', title_standoff=0)
fig.update_xaxes(range=[0.5, 5.5], ticksuffix='°C', title='Temperature', dtick=1)
    
fig.update_layout(
    legend_y=0.5, width=950, height=350, margin={'l': 20, 'r': 30, 'b': 30, 't': 30}
)
fig.show()

In [55]:
fig.write_image('../Paper/img/Burke_damage_functions.png', scale=2)

In [144]:
a = 0.5953827
std = 0.1904833

In [146]:
b1 = 0.1561274
b2 = 1.034638

In [152]:
delta = (b2 - b1) / 2
delta / std

2.306004253391242

In [171]:
(b2 + b1) / 2

0.5953826999999999

In [172]:
b1*1.25, b2*1.25

(0.19515925, 1.2932975)

In [153]:
from scipy.stats import norm

In [170]:
norm.ppf(0.01056)

-2.3058323832897

## Sensitivity runs

In [79]:
def add_fullname(df):
    df.insert(1, 'fullname', df['name'] + ' ' + df['elasmu'] + ' ' + df['minEmissions'].astype(str) + ' ' + df['maxReductParam'])

df_CE, df_CE_indexed = parse_input('experiment_all.csv', withInertia=True)
add_fullname(df_CE)
df_CBA, df_CBA_indexed = parse_input('experiment_allcba.csv', withInertia=True)
add_fullname(df_CBA)
df_CBA_noinertia, df_CBA_indexed_noinertia = parse_input('experiment_allcba.csv', withInertia=False)
add_fullname(df_CBA_noinertia)

df_CE_extra, df_CE_indexed_extra = parse_input('experiment_extra.csv', withInertia=True, allElasmu=True)
add_fullname(df_CE_extra)

df_CBA_extra, df_CBA_indexed_extra = parse_input('experiment_extracba.csv', withInertia=True, allElasmu=True)
add_fullname(df_CBA_extra)

df_CBA_drupp, df_CBA_indexed_drupp = parse_input('experiment_drupp.csv', withInertia=False, allElasmu=True)
add_fullname(df_CBA_drupp)


df_CE_all = pd.concat([df_CE_extra, df_CE])
df_CBA_all = pd.concat([df_CBA_extra, df_CBA])

In [75]:
def get_defaults(
    df,
    defaultSSP=True, defaultDamage=True, defaultTCRE=True, defaultCost=True, defaultPRTP=True,
    defaultElasmu=True, defaultInertia=True, defaultMinEmissions=True, only2100=False
):
    return df[
        ((df['SSP'] == 'SSP2') | (not defaultSSP))
        & ((df['damage'] == 'Howard Total') | (not defaultDamage))
        & ((df['TCRE'] == '0.62') | (not defaultTCRE))
        & ((df['cost_level'] == 'Median') | (not defaultCost))
        & ((df['r'] == '1.5%') | (not defaultPRTP))
        & ((df['elasmu'] == '1.45') | (not defaultElasmu))
        & ((df['maxReductParam'] == '2.2') | (not defaultInertia))
        & ((df['minEmissions'] == -20) | (not defaultMinEmissions))
        & ((df['year'] == 2100) | (not only2100))
    ]

In [61]:
px.line(
    df_CE_extra,
    x='year', y='E', facet_col='elasmu', facet_row='minEmissions', color='maxReductParam', line_group='fullname',
    height=1000
).update_xaxes(range=[2020, 2100])

In [ ]:
param_columns = ['SSP', 'damage', 'TCRE', 'cost_level', 'r']

In [67]:
df_CBA_all['TCRE'].unique()

[0.62, 0.42, 0.82]
Categories (3, object): [0.62, 0.42, 0.82]

In [73]:
df_CBA_all['elasmu'].unique()

array(['0.2', '1.001', '3.0', '1.45'], dtype=object)

In [104]:
fig = make_subplots(rows=1, cols=4, subplot_titles=('Min. emissions', 'Inertia', 'Elasmu', 'Damage function'))
cmap = dict(zip(df_CBA_all['r'].unique(), px.colors.qualitative.Plotly))

# Min. emissions
for trace in px.scatter(
    get_defaults(df_CBA_all, defaultMinEmissions=False, only2100=True),
    x='minEmissions', y='temp', color='r', color_discrete_map=cmap
).data:
    fig.add_trace(trace.update(showlegend=False), row=1, col=1)

# Inertia
for trace in px.scatter(
    get_defaults(df_CBA_all, defaultInertia=False, only2100=True),
    x='maxReductParam', y='temp', color='r', color_discrete_map=cmap
).data:
    fig.add_trace(trace.update(showlegend=False), row=1, col=2)

# Elasmu
for trace in px.scatter(
    get_defaults(df_CBA_all, defaultElasmu=False, defaultPRTP=False, only2100=True),
    x='elasmu', y='temp', color='r', color_discrete_map=cmap
).data:
    fig.add_trace(trace, row=1, col=3)
    
# Damage
for trace in px.scatter(
    get_defaults(df_CBA_all, defaultDamage=False, defaultPRTP=True, only2100=True),
    x='damage', y='temp', color='r', color_discrete_map=cmap
).data:
    fig.add_trace(trace.update(showlegend=False), row=1, col=4)

fig.update_xaxes(type='category')
fig.update_xaxes(col=1, categoryarray= [-20, -10, 0], title='Limit (GtCO2/yr)')
fig.update_xaxes(col=2, categoryorder='category ascending', title='Limit (GtCO2/yr)')
fig.update_xaxes(col=3, categoryorder='category ascending', title='5th, 50th, avg, 95th perc.')
fig.update_yaxes(matches='y', ticksuffix='°C', dtick=0.25)
fig.update_yaxes(col=1, title='Optimal temperature in 2100')
fig.update_layout(legend_title='PRTP:')
fig.show()

In [112]:
tmp = get_defaults(df_CBA_all[df_CBA_all['year'] <= 2100], defaultElasmu=False, defaultPRTP=False)
tmp = tmp[
    ((tmp['elasmu'] == '1.45') & tmp['r'].isin(['0.1%', '3%'])) |
    (tmp['elasmu'].isin(['0.2', '3.0']) & (tmp['r'] == '1.5%'))
]

px.line(
    tmp,
    x='year', y='E', color='r', line_dash='elasmu', color_discrete_map=cmap
)

In [108]:
px.line(
    get_defaults(df_CE_all[df_CE_all['year'] <= 2100], defaultElasmu=False, defaultPRTP=False),
    x='year', y='temp', color='r', line_dash='elasmu', color_discrete_map=cmap
)

In [25]:
px.line(
    df_CBA_extra,
    x='year', y='temp', facet_col='elasmu', facet_row='minEmissions', color='maxReductParam', line_group='fullname',
    height=1000
).update_xaxes(range=[2020, 2100])

## Drupp et al PRTP/elasmu pairs

In [5]:
observations_hansel_etal_fromsvg = np.array([
    [-73.938579,70.306971], [-73.938579,69.533549], [-73.938579,69.533549], [-73.938579,68.85131], [-73.938579,66.621947], [-73.938579,66.621947], [-73.938579,66.621947], [-73.938579,66.621947], [-73.938579,66.621947], [-73.938579,65.43905], [-73.938579,64.438079], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,62.982352], [-73.938579,60.753076], [-73.938579,59.297134], [-73.938579,59.297134], [-73.938579,59.297134], [-73.938579,59.297134], [-73.938579,59.297134], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,55.657369], [-73.938579,48.332644], [-73.938579,33.683065], [-73.938579,55.657369], [-73.938579,48.332644], [-73.938579,48.332644], [-73.89311,62.982352], [-73.89311,62.982352], [-73.89311,59.297134], [-73.620058,55.657369], [-73.301568,66.621947], [-73.301568,62.982352], [-73.301568,62.982352], [-73.301568,62.982352], [-73.301568,62.982352], [-73.301568,62.982352], [-73.301568,62.982352], [-73.301568,62.982352], [-73.301568,61.526498], [-73.301568,59.297134], [-73.301568,55.657369], [-73.301568,55.657369], [-73.301568,55.657369], [-73.301568,55.657369], [-73.301568,55.657369], [-73.301568,55.657369], [-72.619266,63.71017], [-72.619266,55.657369], [-72.300774,62.982352], [-71.936813,60.024911], [-71.936813,55.657369], [-70.617317,68.441838], [-70.617317,66.621947], [-70.617317,66.621947], [-70.617317,62.982352], [-70.617317,62.982352], [-70.617317,62.982352], [-70.617317,62.982352], [-70.617317,62.982352], [-70.617317,62.982352], [-70.617317,62.982352], [-70.617317,62.254385], [-70.617317,61.526498], [-70.617317,61.526498], [-70.617317,60.024911], [-70.617317,59.297134], [-70.617317,59.297134], [-70.617317,59.297134], [-70.617317,55.657369], [-70.617317,55.657369], [-70.617317,55.657369], [-70.617317,55.657369], [-68.615551,59.297134], [-68.615551,57.113332], [-67.250621,69.533549], [-67.250621,68.85131], [-67.250621,66.621947], [-67.250621,66.621947], [-67.250621,66.621947], [-67.250621,66.621947], [-67.250621,65.893954], [-67.250621,65.166072], [-67.250621,63.71017], [-67.250621,62.982352], [-67.250621,62.982352], [-67.250621,62.982352], [-67.250621,62.982352], [-67.250621,62.982352], [-67.250621,62.982352], [-67.250621,62.982352], [-67.250621,60.753076], [-67.250621,59.297134], [-67.250621,59.297134], [-67.250621,59.297134], [-67.250621,59.297134], [-67.250621,55.657369], [-67.250621,55.657369], [-67.250621,55.657369], [-67.250621,55.657369], [-67.250621,51.972324], [-67.250621,48.332644], [-67.250621,48.332644], [-67.250621,33.683065], [-65.9313,59.297134], [-65.9313,57.84128], [-63.929534,62.982352], [-63.929534,60.753076], [-63.929534,55.657369], [-63.929534,41.007814], [-60.562976,68.85131], [-60.562976,68.85131], [-60.562976,67.850251], [-60.562976,66.621947], [-60.562976,66.621947], [-60.562976,66.621947], [-60.562976,64.438079], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,62.982352], [-60.562976,61.117007], [-60.562976,60.753076], [-60.562976,55.657369], [-60.562976,55.657369], [-60.562976,55.657369], [-60.562976,55.657369], [-60.562976,51.972324], [-57.1961,63.71017], [-57.1961,55.657369], [-57.1961,55.657369], [-53.829365,70.306971], [-53.829365,68.441838], [-53.829365,62.982352], [-53.829365,62.982352], [-53.829365,62.982352], [-53.829365,62.982352], [-53.829365,62.982352], [-53.829365,59.297134], [-47.141585,66.621947], [-47.141585,62.982352], [-47.141585,59.297134], [-40.453797,68.85131], [-40.453797,66.621947], [-33.720363,41.007814], [-33.720363,41.007814], [-27.032405,65.166072], [-20.344754,62.982352]
])

prtp_obs = observations_hansel_etal_fromsvg[:,0]
elasmu_obs = -observations_hansel_etal_fromsvg[:,1]
prtp = (prtp_obs-prtp_obs.min()) / prtp_obs.ptp() * 8 / 100
elasmu = (elasmu_obs-elasmu_obs.min()) / elasmu_obs.ptp() * 5

In [6]:
len(observations_hansel_etal_fromsvg)

172

In [7]:
pairs = list(zip(prtp.round(2), elasmu.round(2)))
pairs_dict = {}
for pair in pairs:
    if pair not in pairs_dict:
        pairs_dict[pair] = 1
    else:
        pairs_dict[pair] += 1
        
pairs_df = (
    pd.Series(pairs_dict)
    .sort_values(ascending=False)
    .to_frame('count').reset_index()
    .rename(columns={'level_0': 'PRTP', 'level_1': 'elasmu'})
)

In [8]:
years = np.arange(2020, 2101, 1)
data = GDP(years, 'SSP2')
mean_g = np.mean((data[1:] - data[:-1]) / data[:-1])

In [9]:
mean_g

0.021082911508849216

In [16]:
def get_surrounding(array_sorted, value):
    for i in range(len(array_sorted)-1):
        if array_sorted[i] <= value and value <= array_sorted[i+1]:
            return array_sorted[i], array_sorted[i+1]
    raise Exception()

In [17]:
sdr = prtp + mean_g * elasmu
sdr_sorted = np.sort(sdr)
sdr_dict = dict(zip(sdr, pairs))

sdr_percentiles = np.quantile(sdr, [0.05, 0.5, 0.95])
print('SDR quantiles:', sdr_percentiles)

for q in [0.05, 0.5, 0.95]:
    percentile = np.quantile(sdr_sorted, q)
    surrounding = get_surrounding(sdr_sorted, percentile)
    distance = (percentile - surrounding[0]) / (surrounding[1] - surrounding[0])
    print('{} {:.0%}: {}, {:.0%}: {}'.format(q, 1-distance, sdr_dict[surrounding[0]], distance, sdr_dict[surrounding[1]]))

SDR quantiles: [0.0106066  0.0367528  0.07296051]
0.05 0%: (0.0, 0.25), 100%: (0.0, 0.5)
0.5 50%: (0.0, 1.5), 50%: (0.02, 0.8)
0.95 55%: (0.02, 2.5), 45%: (0.01, 3.0)


In [42]:
fig = go.Figure()
fig.add_histogram(x=sdr_sorted, nbinsx=30)
for a,b in [(0.0, 0.5), (0.0, 1.5), (0.02, 2.5)]:
    r = a+mean_g*b
    fig.add_shape(
        type='line', x0=r, x1=r, y0=0, y1=1, yref='paper'
    ).add_annotation(
        text='({:.0%}-{})'.format(a,b),
        x=r, y=1, yref='paper', ax=0,
        yanchor='bottom', font_size=15,
        arrowwidth=2
    )
    
fig.update_xaxes(title='SDR').update_yaxes(title='Number of experts')
fig.write_image('../Paper/img/Drupp histogram.png', scale=2)
fig.show()

In [15]:
0.03+1.001*mean_g

0.05110399442035806

In [137]:
df_CBA_druppthree, df_CBA_indexed_druppthree = parse_input('experiment_druppthree.csv', withInertia=False, allElasmu=True)
add_fullname(df_CBA_druppthree)

In [138]:
df_CBA_druppthree[df_CBA_druppthree['year'] == 2100][['r', 'elasmu', 'temp']]

,r,elasmu,temp
16,0.02,2.5,2.641991
53,0.0,0.5,1.190802
90,0.0,1.5,1.689536


In [118]:
get_surrounding(sdr_sorted, 0.25628992)

(0.23977978560064128, 0.2697969437340643)

In [112]:
np.sort(sdr)

array([0.02108291, 0.05110012, 0.12667269, 0.12667269, 0.13665586,
       0.2198139 , 0.22979708, 0.23977979, 0.23977979, 0.26979694,
       0.28067386, 0.30573339, 0.37644721, 0.52417289, 0.52417289,
       0.52417289, 0.52417289, 0.52417289, 0.52512376, 0.52913057,
       0.52913057, 0.53415606, 0.53415606, 0.53415606, 0.53415606,
       0.54413877, 0.54413877, 0.54413877, 0.56417301, 0.57415593,
       0.63354374, 0.68566536, 0.73291627, 0.79295037, 0.82232077,
       0.84228666, 0.92366633, 0.93168016, 0.94668863, 1.02106077,
       1.02106077, 1.02106077, 1.02106077, 1.02106077, 1.02106077,
       1.02106077, 1.02106077, 1.02106077, 1.02106077, 1.02106077,
       1.02112864, 1.02112864, 1.02201164, 1.02201164, 1.02201164,
       1.02201164, 1.02201164, 1.02201164, 1.02201164, 1.02350553,
       1.02601845, 1.02601845, 1.02601845, 1.02601845, 1.02601845,
       1.02601845, 1.02601845, 1.03104394, 1.03104394, 1.03104394,
       1.03104394, 1.03104394, 1.03104394, 1.03104394, 1.03600

In [31]:
df_CBA_drupp['r'] = df_CBA_drupp['r'].astype(str)
df_CBA_drupp.loc[df_CBA_drupp['r'] == '3%', 'r'] = '0.03'
df_CBA_drupp['elasmu'] = df_CBA_drupp['elasmu'].astype(float)
df_CBA_drupp['r'] = df_CBA_drupp['r'].astype(float)

In [62]:
_drupp_temps_2100 = df_CBA_drupp.loc[df_CBA_drupp['year'] == 2100].copy()
_drupp_temps_2100['elasmu'] = _drupp_temps_2100['elasmu'].replace({1.001: 1.0})
drupp_temps_2100 = dict(_drupp_temps_2100.set_index(['r', 'elasmu'])['temp'])

In [71]:
drupp_temps_2100_all = []
for pair, temp in drupp_temps_2100.items():
    drupp_temps_2100_all += [temp] * pairs_dict[pair]
drupp_temps_2100_all = np.array(drupp_temps_2100_all)

In [74]:
np.quantile(drupp_temps_2100_all, q=[0.05, 0.5, 0.95])

array([1.19080228, 1.92410654, 2.67967154])

In [83]:
get_defaults(df_CBA_noinertia, defaultPRTP=False, only2100=True, defaultInertia=False)[['fullname', 'temp']]

,fullname,temp
4456,CBA SSP2 damageHowardTotal 0.00062 p50 0.001 1...,1.705791
4493,CBA SSP2 damageHowardTotal 0.00062 p50 0.015 1...,2.202901
4530,CBA SSP2 damageHowardTotal 0.00062 p50 0.03 1....,2.576600


In [86]:
get_defaults(df_CBA_all, defaultPRTP=False, only2100=True, defaultElasmu=False)[['elasmu', 'r', 'temp']]

,elasmu,r,temp
16,0.2,0.1%,1.161929
53,1.001,0.1%,1.509055
90,3.0,0.1%,2.336892
127,0.2,1.5%,1.673512
164,1.001,1.5%,2.036227
349,3.0,1.5%,2.663062
386,0.2,3%,2.182765
423,1.001,3%,2.455006
460,3.0,3%,2.902186
5455,1.45,0.1%,1.724745


In [ ]:
SDR = 

In [ ]:
lastig omdat ze gekoppeld zijn, 

van prtp is heel veel lieratuur, elasmu is altijd rond de 1 of 1.45. om op basis van één expert elicitation een hele range 

In [44]:
px.scatter(
    df_CBA_drupp[df_CBA_drupp['year'] == 2100],
    x='r', y='elasmu', color='temp'
)

In [ ]:
fig = go.Figure()
years = np.arange(2020,2100+1)
for SSP, color in zip(['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5'], colors_PBL):
    fig.add_scatter(x=years, y=baseline_emissions(years, SSP), name=SSP, line_color=color)
fig.update_layout(
    width=500, height=300,
    legend_y=0.5, font_size=16,
    yaxis_title='Emissions (GtCO<sub>2</sub>)',
    margin={'l': 30, 't': 20, 'b': 10}
)
fig.show()

In [ ]:
selection_damages = {
    'Burke (LR)': damages['damageBurkeWithLag']['SSP2'],
    'Howard Total': damages['damageHowardTotal'],
    'DICE': damages['damageDICE']
}
temp_values = np.linspace(0.5, 5, 100)

fig = go.Figure()
for (name, fct), color in zip(selection_damages.items(), colors_PBL):
    fig.add_scatter(x=temp_values, y=[fct(T) for T in temp_values], name=name, line_color=color)
    
fig.update_layout(
    width=570, height=300,
    legend_y=0.5, font_size=16,
    xaxis_ticksuffix='°C',
    yaxis_tickformat=',.0%', yaxis_title='GDP loss',
    margin={'l': 30, 't': 20, 'b': 10}
)
fig.show()